In [17]:
import os
import json
import pandas as pd
from src.datasets.LongMemEvalDataset import LongMemEvalDataset

set = "longmemeval"
type = "short"

results_dir = (
    f"data/results/{set}/{type}/embeddings_ollama_nomic-embed-text_memory_ollama_gemma3:4b_judge_openai_gpt-5-mini"
)
json_files = [f for f in os.listdir(results_dir) if f.endswith(".json")]

results = []
for filename in json_files:
    filepath = os.path.join(results_dir, filename)
    with open(filepath, "r") as f:
        data = json.load(f)
        results.append(data)

df = pd.DataFrame(results)

longmemeval_df = LongMemEvalDataset(type, set).dataset

df.merge(longmemeval_df[["question_id", "question_type"]], on="question_id", how="inner").groupby("question_type").agg(
    answer_is_correct_mean=("answer_is_correct", "mean"), count=("answer_is_correct", "count")
).sort_values(by="answer_is_correct_mean", ascending=False).reset_index()


,question_type,answer_is_correct_mean,count
0,temporal-reasoning,1.000000,1
1,single-session-preference,1.000000,1
2,multi-session,0.500000,2
3,knowledge-update,0.333333,3


In [18]:
longmemeval_df['question_type'].value_counts()

question_type
temporal-reasoning           133
multi-session                133
knowledge-update              78
single-session-user           70
single-session-assistant      56
single-session-preference     30
Name: count, dtype: int64